In [1]:
import extcolors
import pandas as pd
from colormap import rgb2hex
from matplotlib import pyplot as plt
from PIL import Image

In [2]:
import sys, os

sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(os.path.abspath(os.path.join("../scripts")))

In [3]:
import color_extraction as colorExt

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

path_to_dataset = os.getenv('path_to_dataset')

In [5]:
# load the performance data, we only need the game_id data

data = pd.read_csv(path_to_dataset+'performance_data.csv')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   game_id       905 non-null    object 
 1   preview_link  905 non-null    object 
 2   ER            905 non-null    float64
 3   CTR           905 non-null    float64
dtypes: float64(2), object(2)
memory usage: 28.4+ KB


In [10]:
# example of 1 pic
gameid = data.loc[0, 'game_id']
path_to_dir = path_to_dataset + 'Assets/' + gameid + '/'
#path_to_dir
path_to_pic = path_to_dir+'landing_2.jpg'

In [12]:
# colorExt.identify_color_composition(path_to_pic, tolerance=12, limit=10, visualize=True)
colorExt.identify_color_composition(path_to_pic, tolerance=12, limit=10, visualize=False)

,c_code,rgb,occurrence
0,#F9FFFF,"(249, 255, 255)",61593
1,#C4D8D9,"(196, 216, 217)",40416
2,#615B5D,"(97, 91, 93)",24738
3,#232122,"(35, 33, 34)",18858
4,#624F3E,"(98, 79, 62)",16002
5,#896D5F,"(137, 109, 95)",14425
6,#8C8D8F,"(140, 141, 143)",10134
7,#A1B4BA,"(161, 180, 186)",9249
8,#413E47,"(65, 62, 71)",7831
9,#A48F7E,"(164, 143, 126)",5116


In [ ]:
# next we apply this to as many pic as we have and add the results to a csv file
# we only extract the dominant colors